In [1]:
import dash
import plotly.graph_objs as go
import plotly.express as px
import matplotlib.pyplot as plt
from dash import Dash, dcc, html, Input, Output
from IPython.display import display, HTML
import pandas as pd
import pymysql
from sqlalchemy import create_engine, text
import requests
from datetime import datetime
import json


In [2]:
from flask import Flask
from flask import jsonify

In [3]:
app = Flask(__name__)

In [4]:
pymysql.install_as_MySQLdb()

In [5]:
pymysql.install_as_MySQLdb()
HOSTNAME = '192.168.0.120'
PORT = 3308
USERNAME = 'hjkim'
PASSWORD = '123456'
DATABASE = 'outlook'
CHARSET1 = 'utf8'
CHARSET2 = 'utf-8'

# 위 커넥션 정보와 동일하게 입력|
con_str_fmt = 'mysql+mysqldb://{0}:{1}@{2}:{3}/{4}?charset={5}'
con_str = con_str_fmt.format(USERNAME, PASSWORD, HOSTNAME, PORT, DATABASE, CHARSET1)

engine = create_engine(con_str)
conn = engine.connect()

s_date = (pd.Timestamp.today(tz = 'Asia/Seoul') - pd.Timedelta(days = 365)).strftime("%Y-%m-%d")
s_need = (pd.Timestamp.today(tz = 'Asia/Seoul') - pd.Timedelta(days = 365 + 6)).strftime("%Y-%m-%d")

In [6]:
s_date = (pd.Timestamp.today(tz = 'Asia/Seoul') - pd.Timedelta(days = 365)).strftime("%Y-%m-%d")
s_need = (pd.Timestamp.today(tz = 'Asia/Seoul') - pd.Timedelta(days = 365 + 6)).strftime("%Y-%m-%d")

q = f"""
select *
    from carenation_overview
    where 1=1
    and date_at >= "{s_need}"
"""
df = pd.read_sql_query(q, engine)

var_ls = ['posts', 'apps', 'pay', 'cancel', 'service']
t = {}
for v in var_ls:
    t[v] = pd.pivot_table(df, index = ['date_at'], columns = ['job_type'], values = f'num_{v}', aggfunc = 'sum').reset_index() 
    orig_col, new_col = ['care', 'donghaeng', 'housekeeper'], [f'{v}_care', f'{v}_dh', f'{v}_hk']  
    for c in range(len(orig_col)):
        t[v][new_col[c]] = t[v][orig_col[c]].rolling(window = 7).mean()
    
    t[v] = t[v][t[v][f'{v}_care'].notnull()].reset_index(drop = True).copy().rename_axis(None, axis=1).drop(orig_col, axis = 1)

df_f = t['posts'].copy()
for v in var_ls[1:]:
    df_f = df_f.merge(t[v], on = 'date_at', how = 'outer', validate = '1:1')
df_f = df_f.merge(pd.concat([df[df.care_A.notnull()][['date_at']], df[df.care_A.notnull()][['care_A', 'care_B', 'care_C']].rolling(window = 7).mean()], axis = 1),
                  on = 'date_at', how = 'outer', validate = '1:1')\
    .merge(pd.concat([df[df.job_type == 'care'][['date_at']], df[df.job_type == 'care'][['approved_amt', 'canceled_amt', 'net_amt']].rolling(window = 7).mean()], axis = 1),
           on = 'date_at', how = 'outer', validate = '1:1')

df_f = df_f.sort_values(by='date_at')[6:]

In [21]:
data1 = df_f[['date_at','service_care','care_A','care_B','care_C']]
data2 = df_f[['date_at','pay_care','cancel_care','approved_amt','canceled_amt']]
data3 = df_f[['date_at','posts_care','apps_care']]

data2['approved_amt'] = data2['approved_amt'] * 10**(-8)
data2['canceled_amt'] = data2['canceled_amt'] * 10**(-8)

external_scripts = [
{'src': 'https://cnd.tailwindcss.com'}

]


C:\Users\hmcte\AppData\Local\Temp\ipykernel_5648\2728497338.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data2['approved_amt'] = data2['approved_amt'] * 10**(-8)
C:\Users\hmcte\AppData\Local\Temp\ipykernel_5648\2728497338.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data2['canceled_amt'] = data2['canceled_amt'] * 10**(-8)


In [26]:
data2

,date_at,pay_care,cancel_care,approved_amt,canceled_amt
0,2023-09-12,132.285714,27.857143,0.703529,0.149206
1,2023-09-13,133.857143,28.428571,0.716705,0.144827
2,2023-09-14,136.000000,28.142857,0.732769,0.151246
3,2023-09-15,135.857143,28.428571,0.732187,0.153053
4,2023-09-16,133.428571,27.571429,0.710346,0.132748
...,...,...,...,...,...
360,2024-09-06,365.285714,64.714286,1.701510,0.258300
361,2024-09-07,363.285714,63.142857,1.632982,0.250251
362,2024-09-08,358.142857,62.857143,1.567354,0.249192
363,2024-09-09,347.714286,59.714286,1.505714,0.231691


In [ ]:
data2

In [27]:
data2= json.loads(data2.sort_values(by='date_at').to_json(orient='records'))

In [28]:
data2

[{'date_at': 1694476800000,
  'pay_care': 132.2857142857,
  'cancel_care': 27.8571428571,
  'approved_amt': 0.7035288571,
  'canceled_amt': 0.1492064286},
 {'date_at': 1694563200000,
  'pay_care': 133.8571428571,
  'cancel_care': 28.4285714286,
  'approved_amt': 0.7167047143,
  'canceled_amt': 0.144827},
 {'date_at': 1694649600000,
  'pay_care': 136.0,
  'cancel_care': 28.1428571429,
  'approved_amt': 0.7327694286,
  'canceled_amt': 0.1512457143},
 {'date_at': 1694736000000,
  'pay_care': 135.8571428571,
  'cancel_care': 28.4285714286,
  'approved_amt': 0.7321874286,
  'canceled_amt': 0.1530532857},
 {'date_at': 1694822400000,
  'pay_care': 133.4285714286,
  'cancel_care': 27.5714285714,
  'approved_amt': 0.7103457143,
  'canceled_amt': 0.1327484286},
 {'date_at': 1694908800000,
  'pay_care': 133.8571428571,
  'cancel_care': 26.5714285714,
  'approved_amt': 0.7063874286,
  'canceled_amt': 0.1348841429},
 {'date_at': 1694995200000,
  'pay_care': 130.2857142857,
  'cancel_care': 27.57142

In [16]:
data1

,date_at,service_care,care_A,care_B,care_C
0,2023-09-12,508.714286,NaN,NaN,NaN
1,2023-09-13,514.571429,NaN,NaN,NaN
2,2023-09-14,520.714286,NaN,NaN,NaN
3,2023-09-15,525.714286,NaN,NaN,NaN
4,2023-09-16,530.571429,NaN,NaN,NaN
...,...,...,...,...,...
360,2024-09-06,1429.714286,276.571429,181.000000,972.142857
361,2024-09-07,1425.142857,277.285714,179.857143,968.000000
362,2024-09-08,1419.000000,276.571429,178.857143,963.571429
363,2024-09-09,1403.000000,276.285714,174.857143,951.857143


In [22]:
data1= json.loads(data1.sort_values(by='date_at').to_json(orient='records'))

In [23]:
data1

[{'date_at': 1694476800000,
  'service_care': 508.7142857143,
  'care_A': None,
  'care_B': None,
  'care_C': None},
 {'date_at': 1694563200000,
  'service_care': 514.5714285714,
  'care_A': None,
  'care_B': None,
  'care_C': None},
 {'date_at': 1694649600000,
  'service_care': 520.7142857143,
  'care_A': None,
  'care_B': None,
  'care_C': None},
 {'date_at': 1694736000000,
  'service_care': 525.7142857143,
  'care_A': None,
  'care_B': None,
  'care_C': None},
 {'date_at': 1694822400000,
  'service_care': 530.5714285714,
  'care_A': None,
  'care_B': None,
  'care_C': None},
 {'date_at': 1694908800000,
  'service_care': 536.4285714286,
  'care_A': None,
  'care_B': None,
  'care_C': None},
 {'date_at': 1694995200000,
  'service_care': 539.4285714286,
  'care_A': None,
  'care_B': None,
  'care_C': None},
 {'date_at': 1695081600000,
  'service_care': 540.1428571429,
  'care_A': None,
  'care_B': None,
  'care_C': None},
 {'date_at': 1695168000000,
  'service_care': 537.4285714286,
  

In [24]:
for i in range(len(data1)):
        
    data1[i]['date_at'] = datetime.fromtimestamp(data1[i]['date_at']/1000).strftime("%Y-%m-%d")

In [25]:
data1

[{'date_at': '2023-09-12',
  'service_care': 508.7142857143,
  'care_A': None,
  'care_B': None,
  'care_C': None},
 {'date_at': '2023-09-13',
  'service_care': 514.5714285714,
  'care_A': None,
  'care_B': None,
  'care_C': None},
 {'date_at': '2023-09-14',
  'service_care': 520.7142857143,
  'care_A': None,
  'care_B': None,
  'care_C': None},
 {'date_at': '2023-09-15',
  'service_care': 525.7142857143,
  'care_A': None,
  'care_B': None,
  'care_C': None},
 {'date_at': '2023-09-16',
  'service_care': 530.5714285714,
  'care_A': None,
  'care_B': None,
  'care_C': None},
 {'date_at': '2023-09-17',
  'service_care': 536.4285714286,
  'care_A': None,
  'care_B': None,
  'care_C': None},
 {'date_at': '2023-09-18',
  'service_care': 539.4285714286,
  'care_A': None,
  'care_B': None,
  'care_C': None},
 {'date_at': '2023-09-19',
  'service_care': 540.1428571429,
  'care_A': None,
  'care_B': None,
  'care_C': None},
 {'date_at': '2023-09-20',
  'service_care': 537.4285714286,
  'care_A':

In [ ]:
data1= json.loads(data1.to_json(orient='records'))
    
    for i in range(len(data1)):
        
        data[i]['date_at'] = datetime.fromtimestamp(data[i]['date_at']/1000).strftime("%Y-%m-%d")

In [ ]:
app = Flask(__name__)


In [ ]:
data3 = data3.to_json(orient='records')

In [ ]:
data3 = json.loads(data3)


In [ ]:
data3

In [ ]:
for i in range(len(data3)):
    
    data3[i]['date_at'] = datetime.fromtimestamp(data3[i]['date_at']/1000).strftime("%Y-%m-%d")

In [ ]:
data3 = json.loads(data3)

for i in range(len(data3)):
    
    data3[i]['date_at'] = datetime.fromtimestamp(data3[i]['date_at']/1000).strftime("%Y-%m-%d")

In [ ]:
data3

In [ ]:
data3 = json.loads(data3.to_json(orient='records'))

In [ ]:
data3

In [ ]:
def get(data):
   
    data3 = data   
    data3 = json.loads(data3.to_json(orient='records'))

    for i in range(len(data3)):
        
        data3[i]['date_at'] = datetime.fromtimestamp(data3[i]['date_at']/1000).strftime("%Y-%m-%d")     

    return data3


In [ ]:
get(data3)